In [26]:
# Copyright 2018 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [START drive_quickstart]

#*# This file is modified from the following link 
#*# https://github.com/googleworkspace/python-samples/blob/41f82a79bcd67ba4de329126a8fdcc4e3c0826ff/drive/quickstart/quickstart.py
#*# The comments with #*# is added by Yi-Dar, Tang
from __future__ import print_function
import os.path
import io
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.http import MediaFileUpload
from googleapiclient.http import MediaIoBaseUpload




# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']
#*# For more scopes, please read https://developers.google.com/drive/api/v3/about-auth#OAuth2Authorizing
#*# Such as full authority, 


def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)

    # Call the Drive v3 API
    #*# For more information of list, please read 
    #*# 1. https://developers.google.com/drive/api/v3/reference/files/list
    #*# 2. https://developers.google.com/drive/api/v3/search-files
    #*# They will help you to know the kwargs of files().list
    #*# And how to search with specific query
    #*# (such as mimetype, substring in title, modified time after than, etc.)
    results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])
    
    if not items:
        print('No files found.')
    #else:
    #    print('Files:')
    #    for item in items:
    #        print(u'{0} ({1})'.format(item['name'], item['id']))

    # 修改文件
    file_name = 'A.txt'  # 文件名或文件路径
    query = "name = '{}'".format(file_name)
    results = service.files().list(q=query, pageSize=1).execute()
    files = results.get('files', [])

    if not files:
        print('File not found.')
    else:
        file_id = files[0]['id']  
        new_content = "2" #edit here
        media_body = MediaIoBaseUpload(io.BytesIO(new_content.encode('utf-8')), mimetype='text/plain', resumable=True)
        updated_file = service.files().update(fileId=file_id, media_body=media_body).execute()
        print('File updated: {}'.format(updated_file.get('name')))

if __name__ == '__main__':
    main()
# [END drive_quickstart]

File updated: A.txt
